# Examples of how to use CORDIAL-RT for data currantion and analysis of radtiotherpay DIOCM files

Be sure to include a user_config.txt file in your workign directory with the following content: 

user=USERNAME
dicom_file_parent_folder=D:/Anon_dicom_files
database_path=/database.db

In [ ]:
# Screen files in folder (expects patient subfolders)
import cordialrt.screen_files.dicom_files_dataframe
import pandas as pd

df, errors = cordialrt.screen_files.dicom_files_dataframe.open_dicom_files(parent_folder = 'D:/Anon_dicom_files/Examples/') 
display(df[['number_of_studies','number_of_plans','number_of_cts','number_of_structures','number_of_doses','plan_names']])


In [ ]:
#Explore the content of the files thorugh the patient_object
import cordialrt.screen_files.helpers

df['planned_fractions'] = df.patient_object.apply(cordialrt.screen_files.helpers.planned_fractions)
df['main_fractions'] = df.planned_fractions.apply(cordialrt.screen_files.helpers.fractions_main_plan)

display(df[['number_of_studies','number_of_plans','number_of_cts','number_of_structures',
'number_of_doses','planned_fractions','main_fractions']])


In [ ]:
#Combine with RVS data if avilable
df_rvs = pd.read_excel('D:/RVS_data.xls')
df_merged = pd.merge(df, df_rvs[['patient_id','Delivered Dose in Course']], on ='patient_id', how ='left')
df_merged[['planned_fractions','number_of_plans', 'number_of_doses', 'main_fractions','Delivered Dose in Course']]


Patient 1 needs scaling (total course dose is 50 Gy but plan is only 23 fractions)

Patient 2 is ok as it is

Patient 3 needs to have two dose files summed as it has a have sequential boost

patient 4 has no RVS data but we know from plan name that it is ok with 50Gy in 25 fractions - it has multiple dose filess (dose per beam) which are summed


In [ ]:
# Create a new treatment collection in the databse to represent a research project
import cordialrt.database.database as rtdb

with rtdb.DatabaseCall() as db:
    test_treatment_collection_id = db.create_treatment_collection('Example treatment collection')

print(test_treatment_collection_id)


In [ ]:
# Add a treatment to the treatment collection in the database. 
# DICOM files asscociated to the plan names, chosen will be associated with the treatment.

all_errors = list()
with rtdb.DatabaseCall() as db:
    status, error_log = db.add_new_treatment_to_collection_from_plan_names(
                        patient = df_merged.loc[0].patient_object,
                        collection_id =  test_treatment_collection_id,
                        plan_names = df_merged.loc[0].plan_names,
                        treatment_place = 'Centre 1',
                        main_reference_dose = df_merged.loc[0]['Delivered Dose in Course'].round(0).astype(int),
                        boost_reference_dose = 0,
                        main_dose_scale_factor = 25/df_merged.loc[0].main_fractions,
                        boost_dose_scale_factor = 0,
                        )
    all_errors.append([row.patient_id, status, error_log])


In [ ]:
# Add a treatment to the treatment collection in the database. 
# DICOM files asscociated to the plan names, chosen will be associated with the treatment.

all_errors = list()
with rtdb.DatabaseCall() as db:
    status, error_log = db.add_new_treatment_to_collection_from_plan_names(
                        patient = df_merged.loc[2].patient_object,
                        collection_id =  test_treatment_collection_id,
                        plan_names = df_merged.loc[2].plan_names,
                        treatment_place = 'Centre 1',
                        main_reference_dose = 50,
                        boost_reference_dose = 10,
                        main_dose_scale_factor = 1,
                        boost_dose_scale_factor = 1,
                        )
    all_errors.append([row.patient_id, status, error_log])


In [ ]:
# Load treatments from a treatment collection by initiating the Treatment class

import cordialrt.analysis.treatments_from_collection 

treatments = cordialrt.analysis.treatments_from_collection.init_treatments_from_collection(treatment_collection_id = test_treatment_collection_id)


In [ ]:
# Extract information from the databse and the associated DICOM files, using the treatment class
t = treatments[0]

print('Patient id:', t.patient_id)
print('Patient dicompyler DicomParser dose object:', t.get_dose()) # Note if multiple dosefiles are associated with the treatment, these are summed.
print('Max dose in the dose grid:', t.get_max_dose())
print('Treatment laterality:', t.get_latterality())
print('Plan study date:', t.get_study_date())

# Please see the definition of the Treatment class for more funtionality 


In [ ]:
# Use data extraction module

import cordialrt.analysis.data_extraction as crtda

datapoints_input = list()

datapoints_input.append(crtda.DvhDataPoint('ctvn_imn', 'volume', 'abs'))
datapoints_input.append(crtda.DvhDataPoint('ctvn_imn', 'mean', 'abs'))
datapoints_input.append(crtda.DvhDataPoint('ctvn_imn', 'v90', 'rel'))

datapoints_input.append(crtda.DvhDataPoint('heart', 'volume', 'abs'))
datapoints_input.append(crtda.DvhDataPoint('heart', 'mean', 'abs'))


for i,patient_id in enumerate(df.patient_id.unique()[0:3]):
    datapoints_output = crtda.extract_data_from_patient(collection_id = 70, patient_id = patient_id, data_points= datapoints_input)

    print(f'Patient {i}')
    for data_point in datapoints_output:
        print(data_point.roi_standard_name, data_point.name ,data_point.value )
    print('')                                           


In [ ]:
# Extract non-dvh data and make a dataframe

data_frame_rows = list()

datapoints_input.append(crtda.GenericDataPoint('main_reference_dose'))
datapoints_input.append(crtda.GenericDataPoint('ct_manufacturer'))
datapoints_input.append(crtda.GenericDataPoint('laterality'))


for i,patient_id in enumerate(df.patient_id.unique()[0:3]):
    datapoints_output = crtda.extract_data_from_patient(collection_id = 70, patient_id = patient_id, data_points= datapoints_input)
    data_frame_row = {'patient_id' : f'patient_{i}'}

    for data_point in datapoints_output:
        try:
            name = f'{data_point.roi_standard_name}_{data_point.name}' #If datapoint is a DvhDataPoint
        except AttributeError: 
            name = data_point.name # If datapoint is a GenericDataPoint
            
        data_frame_row[f'{name}'] = data_point.value
    
    data_frame_rows.append(data_frame_row)

df_dvh = pd.DataFrame(data_frame_rows)
display(df_dvh)

df_dvh.to_csv('dvh.csv')
df_dvh.to_excel('dvh.xlsx')
df_dvh.to_stata('dvh.tba')


In [ ]:
# Get a list of implemented non-ROI data extraction methods
crtda.NON_ROI_DATAPOINTS


In [ ]:
# Use augmented structure set instead of the original

import cordialrt.database.database as crtdb

with crtdb.DatabaseCall() as db:
    structure_collection_id = db.new_structure_collection_from_folder(collection_name= 'example_dl',
                                                                      folder_path = 'D:/Anon_dicom_files/Augmented_structures/Examples/')


In [ ]:
data_frame_rows = list()
dapoints_input2 = list()

datapoints_input2.append(crtda.DvhDataPoint('heart', 'volume', 'abs'))
datapoints_input2.append(crtda.DvhDataPoint('heart', 'mean', 'abs'))

# Add the datapoints for the new structures in the augmented structure set
datapoints_input2.append(crtda.DvhDataPoint('dl_heart', 'volume', 'abs'))
datapoints_input2.append(crtda.DvhDataPoint('dl_heart', 'mean', 'abs'))
datapoints_input2.append(crtda.DvhDataPoint('dl_LAD', 'volume', 'abs'))
datapoints_input2.append(crtda.DvhDataPoint('dl_LAD', 'mean', 'abs'))

for i,patient_id in enumerate(df.patient_id.unique()[0:3]):
    datapoints_output = crtda.extract_data_from_patient(collection_id = 70, patient_id = patient_id, data_points= datapoints_input2,
                                                        structure_collection_id = 7)
    data_frame_row = {'patient_id' : f'patient_{i}'}

    for data_point in datapoints_output:
        try:
            name = f'{data_point.roi_standard_name}_{data_point.name}' #If datapoint is a DvhDataPoint
        except AttributeError: 
            name = data_point.name # If datapoint is a GenericDataPoint
            
        data_frame_row[f'{name}'] = data_point.value
    
    data_frame_rows.append(data_frame_row)

df_dvh = pd.DataFrame(data_frame_rows)
display(df_dvh)


In [ ]:
# Add a Region Of Interest to the treatment class

t.add_new_roi('Breast')

# This will initiate a ROI class object
breast = t.roi_by_name('Breast')

# If the ROI class object has the same name as a roi in the structure file, the dvh can be calcalted using dicompyler
breast_dvh = breast.calculate_dvh_for_roi_name()

print('Breast volume', breast_dvh.volume)
print('Breast mean dose', breast_dvh.mean)


In [ ]:
test_treatment_collection_id = 70


In [ ]:
# Count structure names in list of treatmetments

import cordialrt.analysis.utils as crtut

structure_names = list()
for t in treatments:
    structure_names.append(t.extract_roi_names_from_new_data())

crtut.structure_count(structure_names)


In [ ]:
# If we want to analyse multiple treatments at once, they may have different names for the same organ. 
# We can create a synonym collection and associate that with our treatment collection.

import cordialrt.database.database as rtdb

with rtdb.DatabaseCall() as db:
    test_synonym_collection_id = db.create_synonym_collection('Test synonym collection')
    db.associate_synonym_collection_with_treatment_colection(treatment_collection_id = test_treatment_collection_id, 
                                                            synonym_collection_id = test_synonym_collection_id)

# We can then add synonyms to the collection
with rtdb.DatabaseCall() as db: 
    db.add_synoym_to_synonym_colection(test_synonym_collection_id, 'or; cor', 'heart')
    db.add_synoym_to_synonym_colection(test_synonym_collection_id, 'cor', 'heart')
    db.add_synoym_to_synonym_colection(test_synonym_collection_id, 'parasternale lk', 'ctvn_imn')
    db.add_synoym_to_synonym_colection(test_synonym_collection_id, 'parasternal ln', 'ctvn_imn')


In [ ]:
# We can now just use the standard name.

t.add_new_roi('ctvp_breast')

breast = t.roi_by_name('ctvp_breast')

# This function will look for all synonyms associated with the ROI name (for this particular treatment collection)
breast_dvh = breast.calculate_dvh_for_priority_synonym()

print('Breast volume', breast_dvh.volume)
print('Breast mean dose', breast_dvh.mean)

